In [1]:
import os.path as osp
from tqdm import tqdm

In [2]:
INP_CSV = '../data/train_SPT-CRo.csv'
OUT_CSV = '../data/train_SPT-CRo.processed.csv'
DATA_PATH = 'data/SPT-CRo'
BEG_SPK_ID = 2000

In [3]:
def count_lines(fname):
    with open(fname, 'rb') as f:
        n = 0
        for block in iter(lambda: f.read(1024 * 1024), b''):
            n += block.count(b'\n')
    return n

# Modify filepath
def modify_filepath(fname):
    dname = fname.split('_')[0]
    fpath = osp.join(DATA_PATH, dname, 'wavs', fname+'.wav')
    return fpath, dname

def modify_ph_string(ph):
    ph = ph.replace('$ ', '')   # remove sentence-leading pauses
    ph = ph.replace(' $', '')   # remove sentence-trailing pauses
    ph = ph.replace(', #', ',') # replace phrase pauses with comma
    ph = ph.replace(' #', ',')  # replace phrase pauses with comma
    ph = ph.replace('-#', ',')
    # ph = ph.replace('#', '')    # replace phrase pauses
    ph = ph.replace('   ', ' ') # replace multiple spaces
    ph = ph.replace('  ', ' ')
    return ph

In [4]:
n_lines = count_lines(INP_CSV)
n_lines

257767

In [5]:
# Poté zpracujte soubor s progress barem
spk_ids = {}
glob_spk_id = BEG_SPK_ID
with open(INP_CSV, 'r') as fr, open(OUT_CSV, 'w') as fw:
    for line in tqdm(fr, total=n_lines):
        line = line.strip()
        fname, ph = line.split('|')

        fpath, spk_name = modify_filepath(fname)

        if spk_name in spk_ids:
           spk_id = spk_ids[spk_name] 
        else:
            spk_id = glob_spk_id
            spk_ids[spk_name] = glob_spk_id
            glob_spk_id += 1
    
        ph = modify_ph_string(ph)

        # Write modified line
        print(f'{fpath}|{ph}|{spk_id}', file=fw)

100%|██████████| 257767/257767 [00:01<00:00, 170160.92it/s]
